In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.feature_selection import SelectKBest, f_classif
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

In [55]:
dataset_names = sns.get_dataset_names()
print(dataset_names, end = "\n")  


['anagrams', 'anscombe', 'attention', 'brain_networks', 'car_crashes', 'diamonds', 'dots', 'dowjones', 'exercise', 'flights', 'fmri', 'geyser', 'glue', 'healthexp', 'iris', 'mpg', 'penguins', 'planets', 'seaice', 'taxis', 'tips', 'titanic']


In [56]:
data = sns.load_dataset("titanic")

In [57]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [58]:
data.shape

(891, 15)

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [60]:
data.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [61]:
data.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [62]:
missing_percent = (data.isnull().sum() / len(data)) * 100
print(missing_percent[missing_percent > 0])

age            19.865320
embarked        0.224467
deck           77.216611
embark_town     0.224467
dtype: float64


In [63]:
if data['age'].isnull().sum() > 0:
    median_age = data['age'].median()
    data['age'].fillna(median_age, inplace=True)
    print(f"✓ Filled {data['age'].isnull().sum()} missing 'age' values with median: {median_age}")

✓ Filled 0 missing 'age' values with median: 28.0


In [64]:
if 'embarked' in data.columns and data['embarked'].isnull().sum() > 0:
    mode_embarked = data['embarked'].mode()[0]
    data['embarked'].fillna(mode_embarked, inplace=True)
    print(f"✓ Filled missing 'embarked' values with mode: {mode_embarked}")

✓ Filled missing 'embarked' values with mode: S


In [65]:
plt.figure(figsize=(10, 6))
sns.heatmap(data.isnull(), cbar=True, yticklabels=False, cmap='viridis')
plt.title('Missing Values Heatmap (Yellow = Missing)')
plt.show()

In [66]:
missing_data = pd.DataFrame({
    'Column': data.columns,
    'Missing_Count': data.isnull().sum(),
    'Missing_Percentage': (data.isnull().sum() / len(data)) * 100
})

In [67]:
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

In [68]:
print("\nMissing Data Summary:")
print(missing_data)


Missing Data Summary:
                  Column  Missing_Count  Missing_Percentage
deck                deck            688           77.216611
embark_town  embark_town              2            0.224467


In [69]:
plt.figure(figsize=(10, 5))
plt.bar(missing_data['Column'], missing_data['Missing_Percentage'], color='coral')
plt.xlabel('Columns')
plt.ylabel('Missing Percentage (%)')
plt.title('Percentage of Missing Values by Column')
plt.xticks(rotation=45)
plt.show()

In [70]:
if 'deck' in data.columns:
    data.drop(['deck'], axis=1, inplace=True)
    print(" Dropped 'Cabin' column (too many missing values)")

 Dropped 'Cabin' column (too many missing values)


In [71]:
print("Missing values in embark_town:", data['embark_town'].isnull().sum())

Missing values in embark_town: 2


In [72]:
print("\nUnique values in Embark_town:")
print(data['embark_town'].value_counts())


Unique values in Embark_town:
embark_town
Southampton    644
Cherbourg      168
Queenstown      77
Name: count, dtype: int64


In [73]:
mode_embark_town = data['embark_town'].mode()[0]
data['embark_town'].fillna(mode_embark_town, inplace=True)

In [74]:
print(f"\n Filled 2 missing 'Embark_town' values with mode: {mode_embark_town}")

# Verify it's fixed
print("\nMissing values after fix:", data['embark_town'].isnull().sum())


 Filled 2 missing 'Embark_town' values with mode: Southampton

Missing values after fix: 0


In [75]:
# Final check across the entire dataset
print("\n--- Final Missing Values Check ---")
print(data.isnull().sum())

# Total missing values
total_missing = data.isnull().sum().sum()
print(f"\nTotal missing values in dataset: {total_missing}")

if total_missing == 0:
    print("All missing values handled! Ready for Step 3.")
else:
    print("Still have missing values - let's fix them!")



--- Final Missing Values Check ---
survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

Total missing values in dataset: 0
All missing values handled! Ready for Step 3.


In [76]:
categorical_cols = data.select_dtypes(include=['object']).columns
print("Categorical columns:")
print(categorical_cols)

# See some examples
for col in categorical_cols:
    print(f"\n{col}: {data[col].unique()}")

Categorical columns:
Index(['sex', 'embarked', 'who', 'embark_town', 'alive'], dtype='object')

sex: ['male' 'female']

embarked: ['S' 'C' 'Q']

who: ['man' 'woman' 'child']

embark_town: ['Southampton' 'Cherbourg' 'Queenstown']

alive: ['no' 'yes']


In [77]:
fig, axes = plt.subplots(1, len(categorical_cols), figsize=(15, 4))

for i, col in enumerate(categorical_cols):
    data[col].value_counts().plot(kind='bar', ax=axes[i], color='skyblue')
    axes[i].set_title(f'{col} Distribution')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Count')

plt.tight_layout()
plt.show()

In [78]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['sex'] = le.fit_transform(data['sex'])

print("✓ Encoded 'sex' column:")
print(f"  Male → 1, Female → 0")
print(data['sex'].value_counts())

✓ Encoded 'sex' column:
  Male → 1, Female → 0
sex
1    577
0    314
Name: count, dtype: int64


In [79]:
remaining_categorical = data.select_dtypes(include=['object']).columns
print(f"\nColumns to one-hot encode: {list(remaining_categorical)}")

# Apply one-hot encoding
data = pd.get_dummies(data, columns=remaining_categorical, drop_first=True)

print("\n✓ One-hot encoding completed!")
print(f"New shape: {data.shape}")
print(f"\nNew columns created:")
print(data.columns.tolist())



Columns to one-hot encode: ['embarked', 'who', 'embark_town', 'alive']

✓ One-hot encoding completed!
New shape: (891, 17)

New columns created:
['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'class', 'adult_male', 'alone', 'embarked_Q', 'embarked_S', 'who_man', 'who_woman', 'embark_town_Queenstown', 'embark_town_Southampton', 'alive_yes']


In [80]:
# Check data types
print("\n--- Data Types After Encoding ---")
print(data.dtypes)

# Confirm no object types remain
object_cols = data.select_dtypes(include=['object']).columns
if len(object_cols) == 0:
    print("\nSuccess! All columns are now numeric.")
else:
    print(f"\nStill have object columns: {list(object_cols)}")

# Display first few rows to see the changes
print("\n--- First 5 rows after encoding ---")
print(data.head())



--- Data Types After Encoding ---
survived                      int64
pclass                        int64
sex                           int64
age                         float64
sibsp                         int64
parch                         int64
fare                        float64
class                      category
adult_male                     bool
alone                          bool
embarked_Q                     bool
embarked_S                     bool
who_man                        bool
who_woman                      bool
embark_town_Queenstown         bool
embark_town_Southampton        bool
alive_yes                      bool
dtype: object

Success! All columns are now numeric.

--- First 5 rows after encoding ---
   survived  pclass  sex   age  sibsp  parch     fare  class  adult_male  \
0         0       3    1  22.0      1      0   7.2500  Third        True   
1         1       1    0  38.0      1      0  71.2833  First       False   
2         1       3    0  26.0     

In [81]:
# Display all current columns
print("Current columns in dataset:")
print(data.columns.tolist())
print(f"\nTotal columns: {data.shape[1]}")
print(f"Total rows: {data.shape[0]}")


Current columns in dataset:
['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'class', 'adult_male', 'alone', 'embarked_Q', 'embarked_S', 'who_man', 'who_woman', 'embark_town_Queenstown', 'embark_town_Southampton', 'alive_yes']

Total columns: 17
Total rows: 891


In [82]:
columns_to_drop = []

if 'PassengerId' in data.columns:
    columns_to_drop.append('PassengerId')

if 'Name' in data.columns:
    columns_to_drop.append('Name')

if 'Ticket' in data.columns:
    columns_to_drop.append('Ticket')
if 'class' in data.columns:
    columns_to_drop.append('class')

if columns_to_drop:
    data.drop(columns_to_drop, axis=1, inplace=True)
    print(f"Dropped irrelevant columns: {columns_to_drop}")

print(f"\nColumns after dropping: {data.shape[1]}")


Dropped irrelevant columns: ['class']

Columns after dropping: 16


In [83]:
target_col = 'survived'  

correlation = data.corr()[target_col].sort_values(ascending=False)

print("\n--- Correlation with Survival ---")
print(correlation)

# Visualize correlation
plt.figure(figsize=(10, 8))
correlation.drop(target_col).plot(kind='barh', color='teal')
plt.title(f'Feature Correlation with {target_col}')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Features')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()


--- Correlation with Survival ---
survived                   1.000000
alive_yes                  1.000000
who_woman                  0.506562
fare                       0.257307
parch                      0.081629
embarked_Q                 0.003650
embark_town_Queenstown     0.003650
sibsp                     -0.035322
age                       -0.064910
embarked_S                -0.149683
embark_town_Southampton   -0.149683
alone                     -0.203367
pclass                    -0.338481
sex                       -0.543351
adult_male                -0.557080
who_man                   -0.557080
Name: survived, dtype: float64


In [84]:
# Correlation heatmap for all features
plt.figure(figsize=(12, 10))
sns.heatmap(data.corr(), annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Correlation Heatmap of All Features')
plt.tight_layout()
plt.show()


In [85]:
# Set a threshold - features below this correlation get removed
correlation_threshold = 0.05

# Find weak features
weak_features = correlation[abs(correlation) < correlation_threshold].index.tolist()

# Remove the target itself from this list
if target_col in weak_features:
    weak_features.remove(target_col)

print(f"\nWeak features (correlation < {correlation_threshold}): {weak_features}")

# Decide whether to drop them
# data.drop(weak_features, axis=1, inplace=True)  # Uncomment to drop



Weak features (correlation < 0.05): ['embarked_Q', 'embark_town_Queenstown', 'sibsp']


In [ ]:
print("\n--- Final Feature Set ---")
print(f"Features remaining: {data.shape[1]}")
print(f"Column names: {data.columns.tolist()}")

# Show first few rows
print("\nFirst 5 rows:")
print(data.head())


--- Final Feature Set ---
Features remaining: 16
Column names: ['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'adult_male', 'alone', 'embarked_Q', 'embarked_S', 'who_man', 'who_woman', 'embark_town_Queenstown', 'embark_town_Southampton', 'alive_yes']

First 5 rows:
   survived  pclass  sex   age  sibsp  parch     fare  adult_male  alone  \
0         0       3    1  22.0      1      0   7.2500        True  False   
1         1       1    0  38.0      1      0  71.2833       False  False   
2         1       3    0  26.0      0      0   7.9250       False   True   
3         1       1    0  35.0      1      0  53.1000       False  False   
4         0       3    1  35.0      0      0   8.0500        True   True   

   embarked_Q  embarked_S  who_man  who_woman  embark_town_Queenstown  \
0       False        True     True      False                   False   
1       False       False    False       True                   False   
2       False        True    False       

In [87]:
# Replace 'survived' with your actual target column name if different
target_column = 'survived'  # or 'Survived' - check your dataset

# Create y (target) - what we want to predict
y = data[target_column]

# Create X (features) - everything except the target
X = data.drop(target_column, axis=1)

print("✓ Successfully split features and target")
print(f"\nFeatures (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")


✓ Successfully split features and target

Features (X) shape: (891, 15)
Target (y) shape: (891,)


In [88]:
# Display features (X)
print("\n--- Features (X) - First 5 rows ---")
print(X.head())
print(f"\nFeature columns: {X.columns.tolist()}")

# Display target (y)
print("\n--- Target (y) - First 10 values ---")
print(y.head(10))

# Check target distribution
print("\n--- Target Distribution ---")
print(y.value_counts())
print(f"\nSurvival rate: {y.mean():.2%}")



--- Features (X) - First 5 rows ---
   pclass  sex   age  sibsp  parch     fare  adult_male  alone  embarked_Q  \
0       3    1  22.0      1      0   7.2500        True  False       False   
1       1    0  38.0      1      0  71.2833       False  False       False   
2       3    0  26.0      0      0   7.9250       False   True       False   
3       1    0  35.0      1      0  53.1000       False  False       False   
4       3    1  35.0      0      0   8.0500        True   True       False   

   embarked_S  who_man  who_woman  embark_town_Queenstown  \
0        True     True      False                   False   
1       False    False       True                   False   
2        True    False       True                   False   
3        True    False       True                   False   
4        True     True      False                   False   

   embark_town_Southampton  alive_yes  
0                     True      False  
1                    False       True  
2      

In [89]:
import matplotlib.pyplot as plt

# Bar plot of survival distribution
plt.figure(figsize=(8, 5))
y.value_counts().plot(kind='bar', color=['salmon', 'lightgreen'])
plt.title('Survival Distribution')
plt.xlabel('Survived (0 = No, 1 = Yes)')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Pie chart
plt.figure(figsize=(6, 6))
y.value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['salmon', 'lightgreen'],
                      labels=['Died (0)', 'Survived (1)'])
plt.title('Survival Rate')
plt.ylabel('')
plt.tight_layout()
plt.show()


In [90]:
# Make sure target is NOT in features
if target_column in X.columns:
    print(" WARNING: Target column found in features! Removing it...")
    X = X.drop(target_column, axis=1)
else:
    print(" Good! Target is not in features (no data leakage)")

# Final confirmation
print(f"\nFinal X shape: {X.shape}")
print(f"Final y shape: {y.shape}")


 Good! Target is not in features (no data leakage)

Final X shape: (891, 15)
Final y shape: (891,)


In [91]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,        # 20% for testing, 80% for training
    random_state=42,      # For reproducibility (same split every time)
    stratify=y            # Maintains the same survival ratio in both sets
)

print(" Data split completed!")
print(f"\nTraining set size: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set size: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")


 Data split completed!

Training set size: 712 samples (79.9%)
Test set size: 179 samples (20.1%)


In [92]:
print("\n--- Training Set ---")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

print("\n--- Test Set ---")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


--- Training Set ---
X_train shape: (712, 15)
y_train shape: (712,)

--- Test Set ---
X_test shape: (179, 15)
y_test shape: (179,)


In [93]:
# Check survival distribution in both sets
print("\n--- Survival Distribution ---")
print(f"Original dataset: {y.mean():.2%} survived")
print(f"Training set: {y_train.mean():.2%} survived")
print(f"Test set: {y_test.mean():.2%} survived")

# Visualize the distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Original distribution
y.value_counts().plot(kind='bar', ax=axes[0], color=['salmon', 'lightgreen'])
axes[0].set_title('Original Data')
axes[0].set_xlabel('Survived')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Died (0)', 'Survived (1)'], rotation=0)

# Training distribution
y_train.value_counts().plot(kind='bar', ax=axes[1], color=['salmon', 'lightgreen'])
axes[1].set_title('Training Set')
axes[1].set_xlabel('Survived')
axes[1].set_ylabel('Count')
axes[1].set_xticklabels(['Died (0)', 'Survived (1)'], rotation=0)

# Test distribution
y_test.value_counts().plot(kind='bar', ax=axes[2], color=['salmon', 'lightgreen'])
axes[2].set_title('Test Set')
axes[2].set_xlabel('Survived')
axes[2].set_ylabel('Count')
axes[2].set_xticklabels(['Died (0)', 'Survived (1)'], rotation=0)

plt.tight_layout()
plt.show()



--- Survival Distribution ---
Original dataset: 38.38% survived
Training set: 38.34% survived
Test set: 38.55% survived


In [94]:
print("--- Feature Scales (Before Scaling) ---")
print(X_train.describe())
plt.figure(figsize=(12, 5))
X_train.boxplot(rot=90)
plt.title('Feature Scales - Before Scaling')
plt.ylabel('Value Range')
plt.tight_layout()
plt.show()


--- Feature Scales (Before Scaling) ---
           pclass         sex         age       sibsp       parch        fare
count  712.000000  712.000000  712.000000  712.000000  712.000000  712.000000
mean     2.308989    0.644663   29.459860    0.492978    0.390449   31.819826
std      0.833563    0.478952   13.034574    1.060720    0.838134   48.059104
min      1.000000    0.000000    0.420000    0.000000    0.000000    0.000000
25%      2.000000    0.000000   22.000000    0.000000    0.000000    7.895800
50%      3.000000    1.000000   28.000000    0.000000    0.000000   14.454200
75%      3.000000    1.000000   36.000000    1.000000    0.000000   31.000000
max      3.000000    1.000000   80.000000    8.000000    6.000000  512.329200


In [96]:
from sklearn.preprocessing import StandardScaler

# Create the scaler
scaler = StandardScaler()

# Fit on training data and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Transform test data (using the same scaler fitted on training data)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling completed!")
print(f"\nX_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")


Feature scaling completed!

X_train_scaled shape: (712, 15)
X_test_scaled shape: (179, 15)


In [97]:
import pandas as pd

# Convert back to DataFrame for easier viewing
X_train_scaled_data = pd.DataFrame(X_train_scaled, columns=X_train.columns)

print("\n--- Feature Scales (After Scaling) ---")
print(X_train_scaled_data.describe())

# Visualize scaled features
plt.figure(figsize=(12, 5))
X_train_scaled_data.boxplot(rot=90)
plt.title('Feature Scales - After Scaling')
plt.ylabel('Standardized Value')
plt.axhline(y=0, color='red', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()



--- Feature Scales (After Scaling) ---
             pclass           sex           age         sibsp         parch  \
count  7.120000e+02  7.120000e+02  7.120000e+02  7.120000e+02  7.120000e+02   
mean  -1.821265e-16 -9.979533e-18  2.120651e-16 -5.613487e-18 -1.621674e-17   
std    1.000703e+00  1.000703e+00  1.000703e+00  1.000703e+00  1.000703e+00   
min   -1.571457e+00 -1.346933e+00 -2.229476e+00 -4.650843e-01 -4.661832e-01   
25%   -3.709448e-01 -1.346933e+00 -5.727156e-01 -4.650843e-01 -4.661832e-01   
50%    8.295675e-01  7.424273e-01 -1.120778e-01 -4.650843e-01 -4.661832e-01   
75%    8.295675e-01  7.424273e-01  5.021060e-01  4.783345e-01 -4.661832e-01   
max    8.295675e-01  7.424273e-01  3.880117e+00  7.082266e+00  6.697610e+00   

               fare    adult_male         alone    embarked_Q    embarked_S  \
count  7.120000e+02  7.120000e+02  7.120000e+02  7.120000e+02  7.120000e+02   
mean  -1.746418e-17 -8.981580e-17  6.985673e-17 -5.239255e-17  1.496930e-16   
std    1.00

In [98]:
# Side-by-side comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Before scaling
X_train.boxplot(ax=axes[0], rot=90)
axes[0].set_title('Before Scaling')
axes[0].set_ylabel('Original Values')

# After scaling
X_train_scaled_data.boxplot(ax=axes[1], rot=90)
axes[1].set_title('After Scaling')
axes[1].set_ylabel('Standardized Values')
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=0.8)

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression

# Create the model with regularization
model = LogisticRegression(
    penalty='l2',           # Ridge regularization
    C=1.0,                  # Regularization strength (smaller = stronger)
    random_state=42,        # For reproducibility
    max_iter=1000           # Maximum iterations to find solution
)

print("Logistic Regression model created!")
print(f"\nModel parameters:")
print(f"  - Penalty: {model.penalty}")
print(f"  - C (inverse regularization): {model.C}")
print(f"  - Max iterations: {model.max_iter}")


✓ Logistic Regression model created!

Model parameters:
  - Penalty: l2
  - C (inverse regularization): 1.0
  - Max iterations: 1000


In [101]:
# Train the model on scaled training data
model.fit(X_train_scaled, y_train)

print("\nModel training completed!")
print("The model has learned patterns from the training data.")



Model training completed!
The model has learned patterns from the training data.


In [102]:
coefficients = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model.coef_[0]
}).sort_values('Coefficient', ascending=False)

print("\n--- Feature Importance (Coefficients) ---")
print(coefficients)


--- Feature Importance (Coefficients) ---
                    Feature  Coefficient
14                alive_yes     4.583078
5                      fare     0.112507
11                who_woman     0.088676
8                embarked_Q    -0.002107
12   embark_town_Queenstown    -0.002107
9                embarked_S    -0.046721
13  embark_town_Southampton    -0.046721
7                     alone    -0.070288
4                     parch    -0.099760
2                       age    -0.113496
1                       sex    -0.138385
3                     sibsp    -0.177419
6                adult_male    -0.224414
10                  who_man    -0.224414
0                    pclass    -0.311397


In [103]:
plt.figure(figsize=(10, 6))
colors = ['green' if x > 0 else 'red' for x in coefficients['Coefficient']]
plt.barh(coefficients['Feature'], coefficients['Coefficient'], color=colors)
plt.xlabel('Coefficient Value')
plt.ylabel('Features')
plt.title('Feature Importance in Predicting Survival')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()

In [104]:
print(f"\n--- Model Intercept ---")
print(f"Intercept: {model.intercept_[0]:.4f}")



--- Model Intercept ---
Intercept: -1.4043


In [105]:
# Check accuracy on training data
train_accuracy = model.score(X_train_scaled, y_train)
print(f"\n--- Training Performance ---")
print(f"Training Accuracy: {train_accuracy:.2%}")



--- Training Performance ---
Training Accuracy: 100.00%


In [106]:
print("\n" + "="*50)
print("MODEL SUMMARY")
print("="*50)
print(f"Algorithm: Logistic Regression")
print(f"Regularization: {model.penalty.upper()}")
print(f"Training samples: {X_train_scaled.shape[0]}")
print(f"Features used: {X_train_scaled.shape[1]}")
print(f"Training accuracy: {train_accuracy:.2%}")
print("="*50)



MODEL SUMMARY
Algorithm: Logistic Regression
Regularization: L2
Training samples: 712
Features used: 15
Training accuracy: 100.00%


In [108]:
# Predict survival (0 or 1)
y_pred = model.predict(X_test_scaled)

# Predict probabilities (confidence scores)
y_pred_proba = model.predict_proba(X_test_scaled)

print("✓ Predictions completed!")
print(f"\nTotal predictions made: {len(y_pred)}")
print(f"\nFirst 10 predictions: {y_pred[:10]}")
print(f"First 10 actual values: {y_test.values[:10]}")


✓ Predictions completed!

Total predictions made: 179

First 10 predictions: [0 0 1 0 1 1 1 0 0 0]
First 10 actual values: [0 0 1 0 1 1 1 0 0 0]


In [109]:
# Show probabilities for first 10 predictions
print("\n--- Prediction Probabilities (First 10) ---")
print("Index | Died Prob | Survived Prob | Prediction | Actual")
print("-" * 60)

for i in range(10):
    died_prob = y_pred_proba[i][0]
    survived_prob = y_pred_proba[i][1]
    prediction = y_pred[i]
    actual = y_test.values[i]
    
    print(f"{i:5d} | {died_prob:9.2%} | {survived_prob:13.2%} | {prediction:10d} | {actual:6d}")



--- Prediction Probabilities (First 10) ---
Index | Died Prob | Survived Prob | Prediction | Actual
------------------------------------------------------------
    0 |    99.74% |         0.26% |          0 |      0
    1 |    99.73% |         0.27% |          0 |      0
    2 |     2.07% |        97.93% |          1 |      1
    3 |    99.78% |         0.22% |          0 |      0
    4 |     0.56% |        99.44% |          1 |      1
    5 |     0.72% |        99.28% |          1 |      1
    6 |     0.49% |        99.51% |          1 |      1
    7 |    99.44% |         0.56% |          0 |      0
    8 |    99.30% |         0.70% |          0 |      0
    9 |    99.59% |         0.41% |          0 |      0


In [110]:
comparison = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_pred
})

# Count outcomes
correct = (y_pred == y_test).sum()
incorrect = (y_pred != y_test).sum()

print(f"\n--- Prediction Summary ---")
print(f"Correct predictions: {correct} ({correct/len(y_test)*100:.1f}%)")
print(f"Incorrect predictions: {incorrect} ({incorrect/len(y_test)*100:.1f}%)")

# Pie chart
plt.figure(figsize=(8, 6))
plt.pie([correct, incorrect], 
        labels=['Correct', 'Incorrect'],
        autopct='%1.1f%%',
        colors=['lightgreen', 'salmon'],
        startangle=90)
plt.title('Prediction Accuracy on Test Set')
plt.tight_layout()
plt.show()


--- Prediction Summary ---
Correct predictions: 179 (100.0%)
Incorrect predictions: 0 (0.0%)


In [111]:
# Compare predicted vs actual distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Actual distribution
y_test.value_counts().plot(kind='bar', ax=axes[0], color=['salmon', 'lightgreen'])
axes[0].set_title('Actual Survival (Test Set)')
axes[0].set_xlabel('Survived')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Died (0)', 'Survived (1)'], rotation=0)

# Predicted distribution
pd.Series(y_pred).value_counts().plot(kind='bar', ax=axes[1], color=['salmon', 'lightgreen'])
axes[1].set_title('Predicted Survival (Test Set)')
axes[1].set_xlabel('Survived')
axes[1].set_ylabel('Count')
axes[1].set_xticklabels(['Died (0)', 'Survived (1)'], rotation=0)

plt.tight_layout()
plt.show()

print("\nActual distribution:", y_test.value_counts().to_dict())
print("Predicted distribution:", pd.Series(y_pred).value_counts().to_dict())



Actual distribution: {0: 110, 1: 69}
Predicted distribution: {0: 110, 1: 69}


In [112]:
# Extract survival probabilities
survival_probs = y_pred_proba[:, 1]

# Histogram of prediction confidence
plt.figure(figsize=(10, 6))
plt.hist(survival_probs, bins=20, color='skyblue', edgecolor='black')
plt.xlabel('Predicted Survival Probability')
plt.ylabel('Number of Passengers')
plt.title('Distribution of Prediction Confidence')
plt.axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Decision Threshold')
plt.legend()
plt.tight_layout()
plt.show()


In [113]:
# Show detailed examples
print("\n--- Detailed Prediction Examples ---\n")

# Get indices of test set
test_indices = y_test.index[:5]

for idx, test_idx in enumerate(test_indices):
    print(f"Passenger #{idx + 1}:")
    print(f"  Features: {X_test.iloc[idx].to_dict()}")
    print(f"  Predicted: {'Survived' if y_pred[idx] == 1 else 'Died'} (confidence: {max(y_pred_proba[idx]):.1%})")
    print(f"  Actual: {'Survived' if y_test.iloc[idx] == 1 else 'Died'}")
    print(f"  Result: {'✓ CORRECT' if y_pred[idx] == y_test.iloc[idx] else '✗ WRONG'}")
    print()



--- Detailed Prediction Examples ---

Passenger #1:
  Features: {'pclass': 3, 'sex': 1, 'age': 24.0, 'sibsp': 2, 'parch': 0, 'fare': 24.15, 'adult_male': True, 'alone': False, 'embarked_Q': False, 'embarked_S': True, 'who_man': True, 'who_woman': False, 'embark_town_Queenstown': False, 'embark_town_Southampton': True, 'alive_yes': False}
  Predicted: Died (confidence: 99.7%)
  Actual: Died
  Result: ✓ CORRECT

Passenger #2:
  Features: {'pclass': 3, 'sex': 1, 'age': 44.0, 'sibsp': 0, 'parch': 1, 'fare': 16.1, 'adult_male': True, 'alone': False, 'embarked_Q': False, 'embarked_S': True, 'who_man': True, 'who_woman': False, 'embark_town_Queenstown': False, 'embark_town_Southampton': True, 'alive_yes': False}
  Predicted: Died (confidence: 99.7%)
  Actual: Died
  Result: ✓ CORRECT

Passenger #3:
  Features: {'pclass': 3, 'sex': 1, 'age': 22.0, 'sibsp': 0, 'parch': 0, 'fare': 7.225, 'adult_male': True, 'alone': True, 'embarked_Q': False, 'embarked_S': False, 'who_man': True, 'who_woman': F

In [114]:
print("\n" + "="*50)
print("PREDICTION SUMMARY")
print("="*50)
print(f"Test set size: {len(y_test)}")
print(f"Correct predictions: {correct} ({correct/len(y_test)*100:.1f}%)")
print(f"Incorrect predictions: {incorrect} ({incorrect/len(y_test)*100:.1f}%)")
print(f"Predicted survived: {(y_pred == 1).sum()}")
print(f"Predicted died: {(y_pred == 0).sum()}")
print("="*50)



PREDICTION SUMMARY
Test set size: 179
Correct predictions: 179 (100.0%)
Incorrect predictions: 0 (0.0%)
Predicted survived: 69
Predicted died: 110


In [115]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("="*50)
print("MODEL PERFORMANCE METRICS")
print("="*50)
print(f"Accuracy:  {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall:    {recall:.2%}")
print(f"F1-Score:  {f1:.2%}")
print("="*50)


MODEL PERFORMANCE METRICS
Accuracy:  100.00%
Precision: 100.00%
Recall:    100.00%
F1-Score:  100.00%


In [116]:
print("\n--- Understanding the Metrics ---\n")
print("Imagine 100 passengers in test set:")
print("  - 40 actually survived")
print("  - 60 actually died")
print()
print("If our model predicts:")
print("  - 30 survived correctly (True Positives)")
print("  - 10 survived incorrectly (False Positives)")
print("  - 10 missed survivors (False Negatives)")
print("  - 50 died correctly (True Negatives)")
print()
print("Then:")
print(f"  Accuracy  = (30+50)/100 = 80%")
print(f"  Precision = 30/(30+10) = 75% (of predicted survivors, 75% were correct)")
print(f"  Recall    = 30/(30+10) = 75% (caught 75% of actual survivors)")



--- Understanding the Metrics ---

Imagine 100 passengers in test set:
  - 40 actually survived
  - 60 actually died

If our model predicts:
  - 30 survived correctly (True Positives)
  - 10 survived incorrectly (False Positives)
  - 10 missed survivors (False Negatives)
  - 50 died correctly (True Negatives)

Then:
  Accuracy  = (30+50)/100 = 80%
  Precision = 30/(30+10) = 75% (of predicted survivors, 75% were correct)
  Recall    = 30/(30+10) = 75% (caught 75% of actual survivors)


In [117]:
from sklearn.metrics import classification_report

print("\n--- Detailed Classification Report ---\n")
print(classification_report(y_test, y_pred, target_names=['Died', 'Survived']))



--- Detailed Classification Report ---

              precision    recall  f1-score   support

        Died       1.00      1.00      1.00       110
    Survived       1.00      1.00      1.00        69

    accuracy                           1.00       179
   macro avg       1.00      1.00      1.00       179
weighted avg       1.00      1.00      1.00       179



In [118]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

print("\n--- Confusion Matrix ---")
print(cm)
print()
print("Interpretation:")
print(f"  True Negatives (Correctly predicted Died):     {cm[0][0]}")
print(f"  False Positives (Incorrectly predicted Survived): {cm[0][1]}")
print(f"  False Negatives (Incorrectly predicted Died):     {cm[1][0]}")
print(f"  True Positives (Correctly predicted Survived):    {cm[1][1]}")



--- Confusion Matrix ---
[[110   0]
 [  0  69]]

Interpretation:
  True Negatives (Correctly predicted Died):     110
  False Positives (Incorrectly predicted Survived): 0
  False Negatives (Incorrectly predicted Died):     0
  True Positives (Correctly predicted Survived):    69


In [119]:
# Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Died', 'Survived'],
            yticklabels=['Died', 'Survived'],
            cbar=True, square=True, linewidths=2)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()


In [120]:
from sklearn.metrics import roc_curve, roc_auc_score

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba[:, 1])
auc_score = roc_auc_score(y_test, y_pred_proba[:, 1])

print(f"\n--- ROC Analysis ---")
print(f"AUC Score: {auc_score:.4f}")
print("\nAUC Interpretation:")
print("  0.90-1.00: Excellent")
print("  0.80-0.90: Good")
print("  0.70-0.80: Fair")
print("  0.60-0.70: Poor")
print("  0.50-0.60: Fail")



--- ROC Analysis ---
AUC Score: 1.0000

AUC Interpretation:
  0.90-1.00: Excellent
  0.80-0.90: Good
  0.70-0.80: Fair
  0.60-0.70: Poor
  0.50-0.60: Fail


In [121]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [122]:
# Calculate training accuracy
train_pred = model.predict(X_train_scaled)
train_accuracy = accuracy_score(y_train, train_pred)

print("\n--- Training vs Test Performance ---")
print(f"Training Accuracy: {train_accuracy:.2%}")
print(f"Test Accuracy:     {accuracy:.2%}")
print(f"Difference:        {abs(train_accuracy - accuracy):.2%}")

if abs(train_accuracy - accuracy) < 0.05:
    print("\n✓ Good! Model generalizes well (< 5% difference)")
elif abs(train_accuracy - accuracy) < 0.10:
    print("\n⚠ Slight overfitting (5-10% difference)")
else:
    print("\n✗ Warning! Significant overfitting (> 10% difference)")

# Visualize comparison
metrics_comparison = pd.DataFrame({
    'Dataset': ['Training', 'Test'],
    'Accuracy': [train_accuracy, accuracy]
})

plt.figure(figsize=(8, 5))
plt.bar(metrics_comparison['Dataset'], metrics_comparison['Accuracy'], 
        color=['skyblue', 'coral'], edgecolor='black', width=0.5)
plt.ylim([0, 1])
plt.ylabel('Accuracy')
plt.title('Training vs Test Accuracy')
plt.axhline(y=train_accuracy, color='blue', linestyle='--', alpha=0.5)
plt.axhline(y=accuracy, color='red', linestyle='--', alpha=0.5)
for i, v in enumerate(metrics_comparison['Accuracy']):
    plt.text(i, v + 0.02, f'{v:.2%}', ha='center', fontweight='bold')
plt.tight_layout()
plt.show()



--- Training vs Test Performance ---
Training Accuracy: 100.00%
Test Accuracy:     100.00%
Difference:        0.00%

✓ Good! Model generalizes well (< 5% difference)


In [123]:
# Which features contributed most to errors?
import pandas as pd

# Create analysis DataFrame
error_analysis = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_pred,
    'Correct': y_pred == y_test.values
})

# Merge with original features
error_analysis = pd.concat([error_analysis, X_test.reset_index(drop=True)], axis=1)

print("\n--- Error Analysis ---")
print(f"\nTotal errors: {(~error_analysis['Correct']).sum()}")
print("\nBreakdown of errors:")
print(error_analysis[~error_analysis['Correct']]['Actual'].value_counts())



--- Error Analysis ---

Total errors: 0

Breakdown of errors:
Series([], Name: count, dtype: int64)


In [124]:
print("\n" + "="*60)
print("FINAL MODEL EVALUATION SUMMARY")
print("="*60)
print(f"Model: Logistic Regression with L2 Regularization")
print(f"\nPerformance Metrics:")
print(f"  • Accuracy:  {accuracy:.2%}")
print(f"  • Precision: {precision:.2%}")
print(f"  • Recall:    {recall:.2%}")
print(f"  • F1-Score:  {f1:.2%}")
print(f"  • AUC Score: {auc_score:.4f}")
print(f"\nConfusion Matrix:")
print(f"  • True Negatives:  {cm[0][0]}")
print(f"  • False Positives: {cm[0][1]}")
print(f"  • False Negatives: {cm[1][0]}")
print(f"  • True Positives:  {cm[1][1]}")
print(f"\nGeneralization:")
print(f"  • Training Accuracy: {train_accuracy:.2%}")
print(f"  • Test Accuracy:     {accuracy:.2%}")
print(f"  • Difference:        {abs(train_accuracy - accuracy):.2%}")
print("="*60)



FINAL MODEL EVALUATION SUMMARY
Model: Logistic Regression with L2 Regularization

Performance Metrics:
  • Accuracy:  100.00%
  • Precision: 100.00%
  • Recall:    100.00%
  • F1-Score:  100.00%
  • AUC Score: 1.0000

Confusion Matrix:
  • True Negatives:  110
  • False Positives: 0
  • False Negatives: 0
  • True Positives:  69

Generalization:
  • Training Accuracy: 100.00%
  • Test Accuracy:     100.00%
  • Difference:        0.00%
